In [1]:
import numpy as np
import skimage.io
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
import bm3d
import os
import matplotlib.pyplot as plt

In [2]:
def bm3d_output(gt_path, noisy_path, noise_factor, result_path):
        
    nf = str(noise_factor).replace(".", "")
    noise_psnr_total = 0
    bm3d_psnr_total  = 0

    noise_ssim_total = 0
    bm3d_ssim_total  = 0
    
    for root, dirs, files in os.walk(gt_path, topdown=False):
        for name in files:
            img = skimage.io.imread(gt_path + name)
            
            noisy_img = skimage.io.imread(noisy_path + nf + '/' + nf + '_' + name)
            noisy_img_normalized = noisy_img/255.0
            
            out = bm3d.bm3d(noisy_img_normalized, noise_factor)       
            out = np.clip(out, 0,1)*255
            out = out.astype('uint8')
            
            noise_psnr_total += compare_psnr(img, noisy_img)
            bm3d_psnr_total += compare_psnr(img, out)
            
            noise_ssim_total += compare_ssim(img, noisy_img, channel_axis=2)
            bm3d_ssim_total += compare_ssim(img, out, channel_axis=2)
            
            skimage.io.imsave(result_path + 'bm3d_' + name, out)
        
    noise_psnr_total /= len(files)
    bm3d_psnr_total /= len(files)
    noise_ssim_total /= len(files)
    bm3d_ssim_total /= len(files)
    
    print("Noise PSNR: ", noise_psnr_total)
    print("BM3D PSNR: ", bm3d_psnr_total)
    print("Noise SSIM: ", noise_ssim_total)
    print("BM3D SSIM: ", bm3d_ssim_total)
    
    return noise_psnr_total, bm3d_psnr_total, noise_psnr_total, noise_ssim_total

In [3]:
noise_factor = 0.3
nf = str(noise_factor).replace(".", "")

In [4]:
mri_test_path = './data/mri_images_gt/bt_gt_test/'
ct_path = './data/ct_images_gt/'
lympho_path = './data/lymphocytes_images_gt/'

mri_test_path_noisy = mri_test_path + '../bt_noisy_test/'
ct_path_noisy = ct_path + '../ct_noisy/'
lympho_path_noisy = lympho_path + '../lymphocytes_noisy/'

result_path_mri = './data/results/bm3d/mri_test/' + nf + '/'
result_path_ct = './data/results/bm3d/ct_test/'  + nf + '/'
result_path_lympho = './data/results/bm3d/lympho_test/' + nf + '/'

os.makedirs(result_path_mri, exist_ok=True)
os.makedirs(result_path_ct, exist_ok=True)
os.makedirs(result_path_lympho, exist_ok=True)

In [5]:
noise_psnr, bm3d_psnr, noise_ssim, bm3d_ssim = bm3d_output(mri_test_path, mri_test_path_noisy, noise_factor, result_path_mri)

Noise PSNR:  14.422042710888006
BM3D PSNR:  20.320853627492458
Noise SSIM:  0.07511282480826852
BM3D SSIM:  0.43155152886948733


In [6]:
noise_psnr, bm3d_psnr, noise_ssim, bm3d_ssim = bm3d_output(ct_path, ct_path_noisy, noise_factor, result_path_ct)

Noise PSNR:  14.101043192896306
BM3D PSNR:  21.751941731593682
Noise SSIM:  0.059877173004565555
BM3D SSIM:  0.564135118451801


In [7]:
noise_psnr, bm3d_psnr, noise_ssim, bm3d_ssim = bm3d_output(lympho_path, lympho_path_noisy, noise_factor, result_path_lympho)

Noise PSNR:  13.807549973934405
BM3D PSNR:  18.39431375533503
Noise SSIM:  0.4114229213028708
BM3D SSIM:  0.4307443040920647
